# PilotNet SNN Example

Network excange module is available as `lava.lib.dl.netx.{hdf5, blocks, utils}`.
* `hdf5` implements automatic network generation.
* `blocks` implements individual layer blocks.
* `utils` implements hdf5 reading utilities. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import InPort, OutPort
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.proc.conv.process import Conv
from lava.proc import io

from lava.lib.dl import netx
from dataset import PilotNetDataset

# Import modules for Loihi2 execution

Check if Loihi2 compiker is available and import related modules.

In [2]:
loihi2_is_available = netx.utils.Loihi2Exec.is_loihi2_available

if loihi2_is_available:
    import logging
    from lava.magma.core.run_configs import Loihi2HwCfg
    from lava.proc import embedded_io as eio
else:
    print("Loihi2 compiler is not available in this system. "
          "This tutorial will execute on CPU backend.")
# loihi2_is_available = False

## Create network block

A network block can be created by simply instantiating `netx.hdf5.Network` with the path of the desired hdf5 network description file.
* The input layer is accessible as `net.in_layer`.
* The output layer is accessible as `net.out_layer`.
* All the constituent layers are accessible as as a list: `net.layers`.

In [3]:
net = netx.hdf5.Network(net_config='network.net', reset_interval=16, reset_offset=1)
print(net)

|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Input     |  200|   66|    3|     |     |     |     |     |False|
|Conv      |   99|   32|   24| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   49|   15|   36| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   24|    7|   48| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   22|    4|   64| 3, 3| 1, 2| 0, 1| 1, 1|    1|False|
|Conv      |   20|    2|   64| 3, 3| 1, 1| 0, 0| 1, 1|    1|False|
|Dense     |    1|    1|  100|     |     |     |     |     |False|
|Dense     |    1|    1|   50|     |     |     |     |     |False|
|Dense     |    1|    1|   10|     |     |     |     |     |False|
|Dense     |    1|    1|    1|     |     |     |     |     |False|


In [4]:
print(f'There are {len(net)} layers in network:')

for l in net.layers:
    print(f'{l.__class__.__name__:5s} : {l.name:10s}, shape : {l.shape}')

There are 10 layers in network:
Input : Process_1 , shape : (200, 66, 3)
Conv  : Process_3 , shape : (99, 32, 24)
Conv  : Process_6 , shape : (49, 15, 36)
Conv  : Process_9 , shape : (24, 7, 48)
Conv  : Process_12, shape : (22, 4, 64)
Conv  : Process_15, shape : (20, 2, 64)
Dense : Process_18, shape : (100,)
Dense : Process_21, shape : (50,)
Dense : Process_24, shape : (10,)
Dense : Process_27, shape : (1,)


In [5]:
num_samples = 201
steps_per_sample = net.reset_interval
readout_offset = (steps_per_sample - 1) + len(net.layers)
num_steps = num_samples * steps_per_sample + 1

## Create Dataset instance
Typically the user would write it or provide it.

In [6]:
full_set = PilotNetDataset(
    path='../data', 
    transform=net.in_layer.transform, # input transform
    visualize=True, # visualize ensures the images are returned in sequence
    sample_offset=10550,
)
train_set = PilotNetDataset(
    path='../data', 
    transform=net.in_layer.transform, # input transform
    train=True,
)
test_set = PilotNetDataset(
    path='../data', 
    transform=net.in_layer.transform, # input transform
    train=False,
)

# Instantiate Dataloader

In [7]:
# dataloader = io.dataloader.StateDataloader(
#     dataset=full_set,
#     interval=steps_per_sample,
# )
dataloader = io.dataloader.SpikeDataloader(dataset=full_set,
                                           interval=steps_per_sample)

Sample: 10550

# Delta Encoder

In [8]:
class FrameDiff(AbstractProcess):
    def __init__(self, shape, interval=1, offset=0) -> None:
        super().__init__(shape=shape)
        self.old_frame = Var(shape=shape)
        self.frame = Var(shape=shape)
        self.interval = Var(shape=(1,), init=interval)
        self.offset = Var(shape=(1,), init=offset)
        self.diff = OutPort(shape=shape)
        self.inp = InPort(shape=shape)

@implements(proc=FrameDiff, protocol=LoihiProtocol)
@requires(CPU)
class PyFrameDiffModel(PyLoihiProcessModel):
    diff = LavaPyType(PyOutPort.VEC_DENSE, np.int32)  #TODO: make it VEC_SPARSE
    inp = LavaPyType(PyInPort.VEC_DENSE, np.int32)
    old_frame = LavaPyType(np.ndarray, np.int32)
    frame = LavaPyType(np.ndarray, np.int32)
    interval = LavaPyType(np.ndarray, np.int32)
    offset = LavaPyType(np.ndarray, np.int32)

    def run_spk(self) -> None:
        self.frame = self.inp.recv()
        if (self.time_step - 1) % self.interval == self.offset:
            self.diff.send((self.frame - self.old_frame))
            self.old_frame = self.frame

# Connect Input and Output

In [9]:
net.in_layer.neuron.du.init = 4095  # Make current state persistent
kernel = np.eye(3).reshape(3, 1, 1, 3)
input_conv = Conv(weight=kernel, input_shape=net.inp.shape)

gt_logger = io.sink.RingBuffer(shape=(1,), buffer=num_steps)
output_logger = io.sink.RingBuffer(shape=net.out.shape, buffer=num_steps)
input_encoder = FrameDiff(shape=net.inp.shape, interval=steps_per_sample)

# input_adapter = eio.state.WriteConv(shape=net.inp.shape, interval=steps_per_sample)
input_adapter = eio.spike.PyToN3ConvAdapter(shape=input_conv.s_in.shape, interval=steps_per_sample, offset=1)
out_adapter = eio.state.Read(shape=net.out.shape)

dataloader.ground_truth.connect(gt_logger.a_in)
# dataloader.connect_var(input_encoder.frame)
dataloader.s_out.connect(input_encoder.inp)
input_encoder.diff.connect(input_adapter.inp)
# input_adapter.connect_var(net.in_layer.neuron.u)
input_adapter.out.connect(input_conv.s_in)
input_conv.a_out.connect(net.in_layer.neuron.a_in)
out_adapter.connect_var(net.out_layer.neuron.v)
out_adapter.out.connect(output_logger.a_in)

In [18]:
# net.layers[2].synapse.weight.init.shape

(36, 3, 3, 24)

## Run

### Customize Run Configuration

In [10]:
# class CustomRunConfig(Loihi1SimCfg):
#     def select(self, proc, proc_models):
#         # customize run config to always use float model for io.sink.RingBuffer
#         if isinstance(proc, io.sink.RingBuffer):
#             return io.sink.PyReceiveModelFloat
#         else:
#             return super().select(proc, proc_models)

### Run the network

In [11]:
# run_config = CustomRunConfig(select_tag='fixed_pt')
net._log_config.level = logging.INFO
run_config = Loihi2HwCfg()
net.run(condition=RunSteps(num_steps=num_steps), run_cfg=run_config)
output = output_logger.data.get().flatten()
gts = gt_logger.data.get().flatten()
net.stop()
output = (output.astype(np.int32) << 8) >> 8  # reinterpret the data as 24 bit signed value
results = output[readout_offset - 1::16]

ValueError: could not broadcast input array from shape (9,) into shape (3,)

## Evaluate Results
Plot and compare the results with the dataset ground truth.

Here, we will also compare the Lava output with a known output for the same sequence on Loihi hardware. The results should match 1:1.

In [ ]:
results = results.flatten()/steps_per_sample/32/64
# results = results[1:] - results[:-1]
loihi = np.load('3x3pred.npy')

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(loihi, linewidth=5, label='Loihi output')
plt.plot(results, label='Lava output')
plt.plot(gts, label='Ground truth')
plt.xlabel(f'Sample frames (+{full_set.sample_offset})')
plt.ylabel('Steering angle (radians)')
plt.legend()

In [ ]:
error = np.sum((loihi - results)**2)
print(f'{error=}')